In [8]:
import pandas as pd
from sqlalchemy import create_engine

# Load the dataset
data = pd.read_csv('https://raw.githubusercontent.com/imomenujung/SQL-Project/main/Data-Analyst/Online-sales-dataset/Online%20Sales%20Data.csv')

# Create a SQLite database
engine = create_engine('sqlite:///sales_data.db')
data.to_sql('sales', con=engine, if_exists='replace', index=True)


240

In [9]:
data

,Transaction ID,Date,Product_Category,Product_Name,Units_Sold,Unit_Price,Total_Revenue,Region,Payment_Method
0,10001,1/1/2024,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,1/2/2024,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,1/3/2024,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,1/4/2024,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,1/5/2024,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal
...,...,...,...,...,...,...,...,...,...
235,10236,8/23/2024,Home Appliances,Nespresso Vertuo Next Coffee and Espresso Maker,1,159.99,159.99,Europe,PayPal
236,10237,8/24/2024,Clothing,Nike Air Force 1 Sneakers,3,90.00,270.00,Asia,Debit Card
237,10238,8/25/2024,Books,The Handmaid's Tale by Margaret Atwood,3,10.99,32.97,North America,Credit Card
238,10239,8/26/2024,Beauty Products,Sunday Riley Luna Sleeping Night Oil,1,55.00,55.00,Europe,PayPal


In [10]:
# Connect to SQLite database
conn = sqlite3.connect('sales_data.db')

# Define SQL queries
queries = {
    "Monthly sales trends": """
        SELECT strftime('%Y-%m', Date) AS Month, SUM(Total_Revenue) AS Total_Sales
        FROM sales
        GROUP BY Month
        ORDER BY Month;
    """,
    "Revenue by category in different regions": """
        SELECT Product_category, region, SUM(total_revenue) AS All_Revenue
        FROM sales
        GROUP BY Product_category, region
        ORDER BY Product_category, All_Revenue DESC;
    """,
    "Sales volume and revenue by payment method": """
        SELECT Payment_Method, COUNT(*) AS Sales_Volume, SUM(total_revenue) AS All_Revenue
        FROM sales
        GROUP BY Payment_Method
        ORDER BY All_Revenue DESC;
    """,
    "Top Selling Product": """
        SELECT product_category, Product_Name, SUM(total_revenue) AS All_Revenue
        FROM sales
        GROUP BY product_category, Product_Name
        ORDER BY product_category, All_Revenue DESC;
    """,
    "Regional performance of top products/categories": """
        SELECT Region, product_category, SUM(Total_revenue) AS All_Revenue
        FROM sales
        GROUP BY Region, product_category
        ORDER BY Region, All_Revenue DESC;
    """
}

In [11]:
# Execute queries and store results in DataFrames
results = {key: pd.read_sql(query, conn) for key, query in queries.items()}
conn.close()


In [12]:
# Display results
for key, df in results.items():
    print(f"=== {key} ===")
    display(df.head())

=== Monthly sales trends ===


,Month,Total_Sales
0,None,80567.85


=== Revenue by category in different regions ===


,Product_Category,Region,All_Revenue
0,Beauty Products,Europe,2621.90
1,Books,North America,1861.93
2,Clothing,Asia,8128.93
3,Electronics,North America,34982.41
4,Home Appliances,Europe,18646.16


=== Sales volume and revenue by payment method ===


,Payment_Method,Sales_Volume,All_Revenue
0,Credit Card,120,51170.86
1,PayPal,80,21268.06
2,Debit Card,40,8128.93


=== Top Selling Product ===


,Product_Category,Product_Name,All_Revenue
0,Beauty Products,Dyson Supersonic Hair Dryer,399.99
1,Beauty Products,La Mer Crème de la Mer Moisturizer,190.00
2,Beauty Products,Chanel No. 5 Perfume,129.99
3,Beauty Products,Tom Ford Black Orchid Perfume,125.00
4,Beauty Products,Sunday Riley Good Genes,105.00


=== Regional performance of top products/categories ===


,Region,Product_Category,All_Revenue
0,Asia,Sports,14326.52
1,Asia,Clothing,8128.93
2,Europe,Home Appliances,18646.16
3,Europe,Beauty Products,2621.90
4,North America,Electronics,34982.41
